# Convert RNMC SQLite files to SPPARKS format
Converts both initial_state.sqlite and reaction_network.sqlite


In [30]:
import sqlite3
import numpy as np
import pandas as pd

In [31]:
def sqlite_to_df(path, column_name):
    '''Take in path to sqlite file and return data frame of specified columns'''
    
    # Read sqlite query results into a pandas DataFrame
    con = sqlite3.connect(path)
    df = pd.read_sql_query("SELECT * FROM " + column_name, con)
    con.close()
    
    return df   

In [32]:
########################### USER_DEFINED ##############################
path_state = '../../LGMC/CO_oxidation/initial_state.sqlite'
path_rn = '../../LGMC/CO_oxidation/rn.sqlite'
file_path_input = 'input_small.txt'
#######################################################################

df_initial_state = sqlite_to_df(path_state, 'initial_state')
df_rn = sqlite_to_df(path_rn, 'reactions')

species = df_initial_state['species_id'].to_numpy()

In [33]:
# create mapping between numerical species of alphabetical names
mapping = {i : chr(int(i/26) + 65) + chr(i%26 + 65) for i in range(np.shape(species)[0])}

In [34]:
# create file with basic chemistry commands

f = open('input_small.txt', 'w')

f.write('# SPPARKS chemistry application \n')
f.write('# RNMC example \n')
f.write('seed 1234 \n')
f.write('app_style chemistry\n')
f.write('solve_style tree \n')
f.write('volume 1.9098395e-11\n')


21

In [35]:
# add species

f = open(file_path_input, 'a')

for i in mapping:
    f.write('add_species ' + mapping[i] + '\n')

In [36]:
# add reactions

f = open(file_path_input, 'a')
for index, row in df_rn.iterrows():
    
    f.write('add_reaction ' + str(index) + ' ')
    
    if(row['number_of_reactants'] == 1):
        f.write(mapping[row['reactant_1']] + ' ')
    else:
        f.write(mapping[row['reactant_1']] + ' ' + mapping[row['reactant_2']] + ' ')
    
    f.write(str(row['rate']) + ' ')
        
    if(row['number_of_products'] == 1):
        f.write(mapping[row['product_1']] + '\n')
    else:
        f.write(mapping[row['product_1']] + ' ' + mapping[row['product_2']] + '\n')

In [37]:
# initial state
f = open(file_path_input, 'a')

for index, row in df_initial_state[df_initial_state['count'] > 0].iterrows():
    f.write('count ' + mapping[row['species_id']] + ' ' + str(row['count']) + '\n')
    
f.close()

In [38]:
for index, row in df_rn.iterrows():
    
    print('add_reaction ' + str(index), end=' ')
    
    if(row['number_of_reactants'] == 1):
        print(mapping[row['reactant_1']], end=' ')
    else:
        print(mapping[row['reactant_1']] + ' ' + mapping[row['reactant_2']], end=' ')
    
    print(str(row['rate']), end=' ')
        
    if(row['number_of_products'] == 1):
        print(mapping[row['product_1']])
    else:
        print(mapping[row['product_1']] + ' ' + mapping[row['product_2']])

add_reaction 0 AA AB 1000000.0 AB
add_reaction 1 AA AE 1000.0 AE
add_reaction 2 AE 1000.0 AA AE
add_reaction 3 AE 0.0 AD
add_reaction 4 AD 0.0 AE
add_reaction 5 AB AD 0.0 AC AA
add_reaction 6 AC 100000.0 AA AC
add_reaction 7 AA AB 1.0 AA AB
add_reaction 8 AA AE 1.0 AA AE
